In [3]:
from datasets import Dataset
import transformers
import torch
import arabert
import dataset
import emoji

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import string
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report as creport
from multiprocessing import Pool


# T5

## Dataset

In [16]:
train_df = pd.read_csv('train_dataset.csv')
test_df = pd.read_csv('test_dataset.csv')
val_df = pd.read_csv('val_dataset.csv')

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
val_dataset = Dataset.from_pandas(val_df)


dataset = {
    'train': train_dataset,
    'val': val_dataset
}

In [17]:
dataset['train'][0]

{'Tweet': 'اليوم يا انا فيني شي يا الناس متسلطه علي ', 'labels': 0}

In [18]:
train_df = pd.DataFrame({
    'id':range(len(train_df)),
    'label':train_dataset["labels"],
    'text': train_dataset["Tweet"]
})

val_df = pd.DataFrame({
    'id':range(len(val_df)),
    'label':val_dataset["labels"],
    'text': val_dataset["Tweet"]
})

test_df = pd.DataFrame({
    'id':range(len(test_df)),
    'label':test_dataset["labels"],
    'text': test_dataset["Tweet"]
})


# Tokenize

In [19]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained('T5-base')

def preprocess_function(examples):
    return tokenizer(examples['Tweet'], truncation=True, padding='max_length', max_length=84)

tokenized_datasets = {split: dataset[split].map(preprocess_function, batched=True) for split in dataset}




Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/12082 [00:00<?, ? examples/s]

Map:   0%|          | 0/1196 [00:00<?, ? examples/s]

# Load

In [20]:
from transformers import T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained('T5-base')

In [21]:
def compute_metrics(p): #p should be of type EvalPrediction
    print(np.shape(p.predictions[0]))
    print(np.shape(p.predictions[1]))
    print(len(p.label_ids))
    preds = np.argmax(p.predictions[0], axis=1)
    assert len(preds) == len(p.label_ids)
    print(classification_report(p.label_ids,preds))
    print(confusion_matrix(p.label_ids,preds))

    macro_f1 = f1_score(p.label_ids,preds,average='macro')
    macro_precision = precision_score(p.label_ids,preds,average='macro')
    macro_recall = recall_score(p.label_ids,preds,average='macro')
    acc = accuracy_score(p.label_ids,preds)
    return {
      'macro_f1' : macro_f1,
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc
    }

In [22]:
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score
import evaluate
from transformers import AutoConfig, BertForSequenceClassification, AutoTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor, InputFeatures
from transformers import Trainer , TrainingArguments
from transformers import T5ForConditionalGeneration
from transformers import  T5ForSequenceClassification
from transformers import T5Tokenizer

# model = T5ForConditionalGeneration(config=config)


In [23]:
from transformers import T5Config

In [24]:
model_name = 'T5-base'
num_labels = 3 

In [25]:
config = T5Config.from_pretrained(model_name,num_labels=num_labels, output_attentions=True)
tokenizer = T5Tokenizer.from_pretrained(model_name,
                                          do_lower_case=False,
                                          do_basic_tokenize=True)
tokenizer.max_len = 84
model = T5ForSequenceClassification.from_pretrained(model_name, config=config)

train_dataset = SingleSentenceClassificationProcessor(mode='classification')
val_dataset = SingleSentenceClassificationProcessor(mode='classification')

train_dataset.add_examples(texts_or_text_and_labels=train_df['text'],labels=train_df['label'],overwrite_examples = True)
val_dataset.add_examples(texts_or_text_and_labels=val_df['text'],labels=val_df['label'],overwrite_examples = True)
print(train_dataset.examples[0])

train_features = train_dataset.get_features(tokenizer = tokenizer, max_length =84)
val_features = val_dataset.get_features(tokenizer = tokenizer, max_length =84)
# print(config)

print(len(train_features))
print(len(val_features))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at T5-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


InputExample(guid=None, text_a='اليوم يا انا فيني شي يا الناس متسلطه علي ', text_b=None, label=0)
12082
1196


In [26]:
training_args = TrainingArguments("./train")
training_args.do_train = True
training_args.evaluate_during_training = True
training_args.adam_epsilon = 1e-8
training_args.learning_rate = 2e-5
training_args.warmup_steps = 0
training_args.per_device_train_batch_size = 16
training_args.per_device_eval_batch_size = 2
training_args.num_train_epochs = 5
training_args.logging_steps = 100
training_args.save_steps = 500
training_args.seed = 42
print(training_args.logging_steps)

100


In [27]:
import os
import gc
torch.cuda.empty_cache()
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

torch.cuda.empty_cache()

gc.collect()  # Run garbage collection
torch.cuda.empty_cache()  # Clear the GPU cache

torch.cuda.reset_max_memory_allocated()
torch.cuda.reset_max_memory_cached()



C:\Users\Mohamed\anaconda3\Lib\site-packages\torch\cuda\memory.py:329: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(
C:\Users\Mohamed\anaconda3\Lib\site-packages\torch\cuda\memory.py:356: FutureWarning: torch.cuda.reset_max_memory_cached now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


In [29]:
# instantiate trainer
trainer = Trainer(model=model,
                  args = training_args,
                  train_dataset = train_features,
                  eval_dataset = val_features,
                  compute_metrics = compute_metrics,
)


In [16]:
trainer.train()

Step,Training Loss
100,0.917800
200,0.885000
300,0.871300
400,0.861200
500,0.845600
600,0.859900
700,0.884600
800,0.862700
900,0.867300
1000,0.840300


TrainOutput(global_step=3780, training_loss=0.8588947114490327, metrics={'train_runtime': 1368.2903, 'train_samples_per_second': 44.15, 'train_steps_per_second': 2.763, 'total_flos': 6053442950716560.0, 'train_loss': 0.8588947114490327, 'epoch': 5.0})

In [17]:
model.save_pretrained('./fine-tuned-arat5_v3')
# Save the tokenizer
tokenizer.save_pretrained('./fine-tuned-arat5_v3')

('./fine-tuned-arat5_v3\\tokenizer_config.json',
 './fine-tuned-arat5_v3\\special_tokens_map.json',
 './fine-tuned-arat5_v3\\spiece.model',
 './fine-tuned-arat5_v3\\added_tokens.json')

In [18]:
torch.save(model.state_dict(), 'arat5model_v3.pth')

In [19]:
torch.save(model, 'arat5model_complete_v3.pth')


In [30]:
model = torch.load('arat5model_complete_v3.pth')
model.eval()

T5ForSequenceClassification(
  (transformer): T5Model(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=768, out_features=3072, bias=False)
                (wo): Linear(in_feat

In [31]:
import os

torch.cuda.empty_cache()
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'


In [32]:

trainer.evaluate()

OutOfMemoryError: CUDA out of memory. Tried to allocate 164.00 MiB. GPU 0 has a total capacty of 8.00 GiB of which 0 bytes is free. Of the allocated memory 14.04 GiB is allocated by PyTorch, and 393.45 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import tqdm

print(torch.cuda.is_available())
device = "cuda" if torch.cuda.is_available() else "cpu"

test_dataset = SingleSentenceClassificationProcessor(mode='classification')

test_dataset.add_examples(texts_or_text_and_labels=test_df['text'],labels=test_df['label'],overwrite_examples = True)
print(test_dataset.examples[0])

test_features = test_dataset.get_features(tokenizer = tokenizer, max_length =64)


input_ids = [i.input_ids for i in test_features]
attention_masks = [i.attention_mask for i in test_features]

# Convert all of our data into torch tensors, the required datatype for our model
inputs = torch.tensor(input_ids)
masks = torch.tensor(attention_masks)

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop,
# with an iterator the entire dataset does not need to be loaded into memory
batch_size = 64
test_data = TensorDataset(inputs, masks)

# do not use shuffle, we need the preds to be in same order
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [ ]:
# Put the model in an evaluation state
model.eval()

# Transfer model to GPU
model.to(device)

outputs = []
for input, masks in test_dataloader:
    torch.cuda.empty_cache() # empty the gpu memory

    # Transfer the batch to gpu
    input = input.to(device)
    masks = masks.to(device)

    # Run inference on the batch
    output = model(input, attention_mask=masks)["logits"]
    predictions = torch.argmax(output, dim=1)
    # Transfer the output to CPU again and convert to numpy
    predictions = predictions.cpu().detach().numpy()

    # Store the output in a list
    outputs.extend(predictions)

# Concatenate all the lists within the list into one list
output_labels = test_df["label"]

from sklearn.metrics import f1_score, classification_report

print(classification_report(test_df["label"].values, outputs, target_names=["NOT_OFF,NOT_HS", "OFF,NOT_HS","OFF,HS"]))